In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

file_dir = './combined_milk final.csv'
time_dir = './time.csv'
data = pd.read_csv(file_dir)
calendar = pd.read_csv(time_dir)

In [2]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
#change to 1.0.1

The scikit-learn version is 1.4.1.post1.


In [3]:
data

,Time_ID,Year,SKU,Sales,Discount,Price,Display1,Display2,Feature1,Feature2,Feature3,Feature4,Store_ID
0,1114,2001,SS1,5.000000,0,4.000000,0,0,0,0,0,0,234212
1,1115,2001,SS1,5.000000,0,4.290000,0,0,0,0,0,0,234212
2,1116,2001,SS1,8.000000,0,4.290000,0,0,0,0,0,0,234212
3,1117,2001,SS1,4.000000,0,4.290000,0,0,0,0,0,0,234212
4,1118,2001,SS1,10.000000,0,4.290000,0,0,0,0,0,0,234212
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66142,1470,2007,88_6_99998_59549,1262.061093,0,3.067416,0,0,0,0,0,0,657979
66143,1471,2007,88_6_99998_59549,1262.061093,0,3.067416,0,0,0,0,0,0,657979
66144,1472,2007,88_6_99998_59549,1262.061093,0,3.067416,0,0,0,0,0,0,657979
66145,1473,2007,88_6_99998_59549,1262.061093,0,3.067416,0,0,0,0,0,0,657979


In [4]:
calendar

,IRI Week,Calendar week starting on,Calendar week ending on,Jan,Feb,Mar,Apr,May,Jun,July,...,President,President_1,Easter,Easter_1,Memorial,Memorial_1,4thJuly,4thJuly_1,Labour,Labour_1
0,1114,1-Jan-01,7-Jan-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1115,8-Jan-01,14-Jan-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1116,15-Jan-01,21-Jan-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1117,22-Jan-01,28-Jan-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1118,29-Jan-01,4-Feb-01,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1474,26-Nov-07,2-Dec-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,1475,3-Dec-07,9-Dec-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,1476,10-Dec-07,16-Dec-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,1477,17-Dec-07,23-Dec-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Get sku data for a particular store
store_id_input = 236117
sku_id = '0_1_25293_60025'
# sku_id = '0_1_49900_27268'
store_df = data[data['Store_ID'] == store_id_input]
store_sku_df = store_df[store_df['SKU'] == sku_id]

In [6]:
store_sku_df

,Time_ID,Year,SKU,Sales,Discount,Price,Display1,Display2,Feature1,Feature2,Feature3,Feature4,Store_ID
14600,1114,2001,0_1_25293_60025,4.0,0,1.69,0,0,0,0,0,0,236117
14601,1115,2001,0_1_25293_60025,10.0,0,1.69,0,0,0,0,0,0,236117
14602,1116,2001,0_1_25293_60025,16.0,0,1.69,0,0,0,0,0,0,236117
14603,1117,2001,0_1_25293_60025,19.0,0,1.69,0,0,0,0,0,0,236117
14604,1118,2001,0_1_25293_60025,18.0,0,1.69,0,0,0,0,0,0,236117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,1474,2007,0_1_25293_60025,7.0,0,2.99,0,0,0,0,0,0,236117
14961,1475,2007,0_1_25293_60025,8.0,0,2.99,0,0,0,0,0,0,236117
14962,1476,2007,0_1_25293_60025,11.0,0,2.99,0,0,0,0,0,0,236117
14963,1477,2007,0_1_25293_60025,6.0,0,2.99,0,0,0,0,0,0,236117


### Competitor Data

In [7]:
# Get competitor df
store_compet_sku_df = store_df[store_df['SKU'] != sku_id]

# Competitor sku list
compet_sku = store_df[store_df['SKU'] != sku_id]['SKU'].tolist()
compet_sku = list(OrderedDict.fromkeys(compet_sku))

store_compet_sku_df = store_compet_sku_df.pivot_table(index = ['Time_ID', 'Year', 'Store_ID'], columns = 'SKU', 
                                                      values = ['Price', 'Sales', 'Display1', 'Display2', 'Feature1', 'Feature2', 'Feature3', 'Feature4'])

store_compet_sku_df.columns = ['_'.join([col[1], col[0]]) for col in store_compet_sku_df.columns]
store_compet_sku_df.reset_index(inplace=True)

In [8]:
store_compet_sku_df

,Time_ID,Year,Store_ID,0_1_25293_60027_Display1,0_1_25293_60039_Display1,0_1_28000_58454_Display1,0_1_49900_27268_Display1,0_2_25293_60037_Display1,0_2_28000_24610_Display1,7_1_42365_22800_Display1,...,88_6_99998_59525_Sales,88_6_99998_59546_Sales,88_6_99998_59547_Sales,88_6_99998_59548_Sales,88_6_99998_59549_Sales,88_6_99998_59597_Sales,88_6_99998_59601_Sales,88_6_99998_59649_Sales,88_6_99998_59651_Sales,88_6_99998_59689_Sales
0,1114,2001,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,469.000000,52.000000,146.000000,288.00000,429.000000,22.000000,13.000000,15.00000,30.000000,6.00000
1,1115,2001,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,456.000000,38.000000,114.000000,205.00000,383.000000,18.000000,26.000000,13.00000,21.000000,18.00000
2,1116,2001,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,459.000000,46.000000,120.000000,233.00000,368.000000,39.000000,17.000000,28.00000,27.000000,18.00000
3,1117,2001,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,453.000000,62.000000,132.000000,204.00000,344.000000,32.000000,17.000000,30.00000,35.000000,23.00000
4,1118,2001,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,432.000000,51.000000,136.000000,181.00000,365.000000,23.000000,27.000000,29.00000,31.000000,20.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1474,2007,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,481.220447,34.044728,84.677316,174.86262,308.722045,20.592834,14.715655,20.13099,29.878594,13.61039
361,1475,2007,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,481.220447,34.044728,84.677316,174.86262,308.722045,20.592834,14.715655,20.13099,29.878594,13.61039
362,1476,2007,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,481.220447,34.044728,84.677316,174.86262,308.722045,20.592834,14.715655,20.13099,29.878594,13.61039
363,1477,2007,236117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,481.220447,34.044728,84.677316,174.86262,308.722045,20.592834,14.715655,20.13099,29.878594,13.61039


In [9]:
def price_discount(df, sku = None, year_col='Year'):
    out_df = df.copy()
    price_col = f'{sku}_Price' if sku is not None else 'Price'
    # Calculate the lower bound as 95% of the maximum price within each year
    lower_bound = out_df.groupby(year_col)[price_col].transform(lambda x: np.max(x) * 0.95)
    # Filter the DataFrame based on the condition
    filtered_df = out_df[out_df[price_col] >= lower_bound]
    # Calculate the median for each group of time_year in the filtered DataFrame
    median_by_time_year = filtered_df.groupby(year_col)[price_col].median()
    # Copy the median values back into the original DataFrame by year_col
    out_df.loc[:,'median_price'] = out_df[year_col].map(median_by_time_year)
    # Compute the relative discount
    pc_disc =  out_df['median_price'] / out_df[price_col]
    
    
    z_scores = (pc_disc - np.mean(pc_disc)) / np.std(pc_disc)
    # Identify indices where the absolute z-score is greater than or equal to 3
    outlier_indices = np.where(np.abs(z_scores) >= 3)[0]
    # Replace outliers with the maximum value from X_pc_d excluding those outliers
    if len(outlier_indices) > 0:
        pc_disc[outlier_indices] = np.max(pc_disc[~np.isin(np.arange(len(pc_disc)), outlier_indices)])


    # Update Price column
    out_df.loc[:, price_col] = pc_disc

    return out_df

In [10]:
def sales_lag(df, sku = None, neg = True):
    out_df = df.copy()
    sales_col = f'{sku}_Sales' if sku is not None else 'Sales'
    out_df[sales_col] = -np.log(out_df[sales_col].shift(1)) if neg is True else np.log(out_df[sales_col].shift(1))
    return out_df

In [11]:
def sum_columns(df, sku = None, promotype = 'Feature', neg = True):
    out_df = df.copy()
    columns_to_max = [col for col in out_df.columns if col.startswith(sku+'_'+promotype)] if sku is not None else [col for col in out_df.columns if col.startswith(promotype)]
    if not columns_to_max:
        # print(f"No columns found with prefix '{sku}_{type}'")
        return out_df
    # Calculate the maximum values using numpy
    sum_values = np.sum(out_df[columns_to_max].values, axis=1)
    # Create a new column with the maximum values
    max_column_name = f'{sku}_{promotype}neg' if neg is True else f'{promotype}'
    out_df[max_column_name] = -sum_values if neg is True else sum_values
    # Drop the columns used in the max calculation
    out_df.drop(columns=columns_to_max, inplace=True)
    return out_df


store_compet_final = store_compet_sku_df.copy()
for sku in compet_sku:
    store_compet_final = price_discount(store_compet_final, sku)
    store_compet_final = sum_columns(store_compet_final, sku, 'Feature')    # Negative Features
    store_compet_final = sum_columns(store_compet_final, sku, 'Display')    # Negative Display
    store_compet_final = sales_lag(store_compet_final, sku)                 # Negative Lag Sales 

drop_cols = ['Store_ID', 'median_price']
store_compet_final.drop(columns = drop_cols, inplace = True)

In [54]:
store_compet_final

,Time_ID,Year,0_1_25293_60027_Price,0_1_25293_60039_Price,0_1_28000_58454_Price,0_1_49900_27268_Price,0_2_25293_60037_Price,0_2_28000_24610_Price,7_1_42365_22800_Price,7_1_42365_22820_Price,...,88_6_99998_59597_Featureneg,88_6_99998_59597_Displayneg,88_6_99998_59601_Featureneg,88_6_99998_59601_Displayneg,88_6_99998_59649_Featureneg,88_6_99998_59649_Displayneg,88_6_99998_59651_Featureneg,88_6_99998_59651_Displayneg,88_6_99998_59689_Featureneg,88_6_99998_59689_Displayneg
0,1114,2001,1.000000,1.000000,1.000000,1.330709,1.471405,1.058997,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,1115,2001,1.000000,1.000000,1.000000,1.330709,1.471405,1.058997,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2,1116,2001,1.000000,1.000000,1.000000,1.330709,1.471405,1.058997,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-1.0,-0.0,-0.0,-0.0
3,1117,2001,1.000000,1.000000,1.000000,1.330709,1.471405,1.058997,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-1.0,-0.0,-0.0,-0.0
4,1118,2001,1.000000,1.000000,1.000000,1.310078,1.471405,1.058997,1.0,1.0,...,-0.0,-0.0,-1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1474,2007,1.011792,1.015385,1.000000,1.000000,1.000000,1.147591,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
361,1475,2007,1.023866,1.023866,1.220736,1.000000,1.000000,1.089100,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
362,1476,2007,1.023866,1.023866,1.251429,1.000000,1.000000,1.043027,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
363,1477,2007,1.023866,1.023866,1.000000,1.000000,1.000000,1.113251,1.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


### Store Data

In [32]:
window_size = 8
store_sku_df.reset_index(drop = True, inplace= True)

In [33]:
# SKU Features
store_sku_df_part = store_sku_df.copy()
store_sku_df_part = price_discount(store_sku_df_part)                                                                           # Price Column
store_sku_df_part = sum_columns(store_sku_df_part, promotype = 'Feature', neg = False)                                          # Feature Column
store_sku_df_part = sum_columns(store_sku_df_part, promotype = 'Display', neg = False)                                          # Display Column

store_sku_df_part['Pricelag'] = -store_sku_df_part['Price'].shift(1)                                                            # Negative Price Lag Column
store_sku_df_part['Featurelag'] = -store_sku_df_part['Feature'].shift(1)                                                        # Negative Feature Lag Column
store_sku_df_part['Displaylag'] = -store_sku_df_part['Display'].shift(1)                                                        # Negative Display Lag Column

store_sku_df_part['Saleslag'] = -np.log(store_sku_df_part['Sales'].shift(1))                                                    # Negative Log Sales Lag Column 
store_sku_df_part['Sales_mov_avg'] = np.log(store_sku_df_part['Sales'].rolling(window = window_size).mean()).shift(1)           # Log Sales Rolling Mean Lag Column
store_sku_df_part['Sales'] = np.log(store_sku_df_part['Sales'])                                                                 # Process Sales for Target Variable

In [34]:
# Add Special Events
event_col = ['Halloween', 'Thanksgiving', 'Christmas', 'NewYear', 'President', 'Easter', 'Memorial', '4thJuly', 'Labour']
event_cols = [col if col == 'NewYear' else [col, f'{col}_1'] for col in event_col]
event_cols = [item for sublist in event_cols for item in ([sublist] if isinstance(sublist, str) else sublist)]
calendar_cols = calendar[['IRI Week']+ event_cols]
calendar_cols = calendar_cols.fillna(0).astype(int)

# Join Special Events to SKU Store Data
store_sku_df_part = pd.merge(store_sku_df_part, calendar_cols, left_on='Time_ID', right_on='IRI Week', how='left')

drop_cols = ['Discount','Store_ID', 'median_price', 'SKU', 'IRI Week']
store_sku_df_part.drop(columns = drop_cols, inplace = True)

In [35]:
store_sku_df_part['Sales_mov_avg']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
360    2.063693
361    2.031432
362    2.079442
363    2.154665
364    2.094946
Name: Sales_mov_avg, Length: 365, dtype: float64

### ElasticNetCV

In [49]:
model_1 = ElasticNetCV(l1_ratio = 1, cv = 20, max_iter = 1000, positive=True)
model_2 = ElasticNetCV(l1_ratio = 1, cv = 20, max_iter = 1000)

# Filter Datasets
year_from = 2001
year_to = 2005
year_test = 2006


store_sku_part_trg = store_sku_df_part[(store_sku_df_part["Year"] >= year_from) & (store_sku_df_part["Year"] <= year_to)]
store_sku_part_trg = store_sku_part_trg.iloc[window_size:]
store_sku_part_test = store_sku_df_part[(store_sku_df_part["Year"] == year_test)]

store_compet_trg = store_compet_final[(store_compet_final["Year"] >= year_from) & (store_sku_df_part["Year"] <= year_to)]
store_compet_trg = store_compet_trg.iloc[window_size:]
store_compet_test = store_compet_final[(store_compet_final["Year"] == year_test)]



# Target Variable
sku_sales_train = store_sku_part_trg['Sales']
sku_sales_test = store_sku_part_test['Sales']

# Feature Variables
feature_list = []
sku_train_drop = ['Time_ID', 'Year', 'Sales']
compet_train_drop = ['Time_ID', 'Year']
store_sku_part_trg = store_sku_part_trg.drop(columns = sku_train_drop)
store_sku_part_test = store_sku_part_test.drop(columns = sku_train_drop)
store_compet_trg = store_compet_trg.drop(columns = compet_train_drop)
store_compet_test = store_compet_test.drop(columns = compet_train_drop)

In [50]:
store_sku_part_trg

,Price,Feature,Display,Pricelag,Featurelag,Displaylag,Saleslag,Sales_mov_avg,Halloween,Halloween_1,...,President,President_1,Easter,Easter_1,Memorial,Memorial_1,4thJuly,4thJuly_1,Labour,Labour_1
8,1.384673,0,0,-1.384673,-0.0,-0.0,-2.833213,2.708050,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1.311410,0,0,-1.384673,-0.0,-0.0,-2.564949,2.780371,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1.311410,0,0,-1.311410,-0.0,-0.0,-3.135494,2.876386,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1.311410,0,0,-1.311410,-0.0,-0.0,-2.944439,2.897292,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1.311410,0,0,-1.311410,-0.0,-0.0,-3.091042,2.917771,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,1.206343,0,0,-1.206343,-0.0,-0.0,-2.474501,2.474501,0,0,...,0,0,0,0,0,0,0,0,0,0
257,1.206343,0,0,-1.206343,-0.0,-0.0,-2.474501,2.474501,0,0,...,0,0,0,0,0,0,0,0,0,0
258,1.206343,0,0,-1.206343,-0.0,-0.0,-2.474501,2.474501,0,0,...,0,0,0,0,0,0,0,0,0,0
259,1.206343,0,0,-1.206343,-0.0,-0.0,-2.474501,2.474501,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
positive_features_1 = ['Price', 'Feature', 'Display', 'Pricelag' ,'Featurelag', 'Displaylag'] # , 'Saleslag'
positive_mask_1 = [feature in positive_features_1 for feature in store_sku_part_trg.columns]
positive_mask_1_1 = np.array(positive_mask_1, dtype=np.bool_)
#the positive parameter only takes one bool value. not a list...

model_1.positive = True
model_1.fit(store_sku_part_trg, sku_sales_train)
sku_sales_train_rsd = sku_sales_train - model_1.predict(store_sku_part_trg)

In [52]:
positive_mask_1_1

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [39]:
# Second Stage Estimation - Parameters for Cross SKU cannibalisation
#positive_features_2 = [col for col in store_compet_trg.columns if col.endswith(("_Displayneg", "_Featureneg", "_Price"))]
#model_2.positive = positive_features_2
model_2.fit(store_compet_trg, sku_sales_train_rsd)
sku_sales_overall_rsd = sku_sales_train_rsd - model_2.predict(store_compet_trg)
adjustment = np.mean(np.exp(sku_sales_overall_rsd))

/Users/emmy/Desktop/Last_term_SMU/kaggle_comp/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.022273118104148892, tolerance: 0.004935986429691189
  model = cd_fast.enet_coordinate_descent_gram(
/Users/emmy/Desktop/Last_term_SMU/kaggle_comp/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05319783401439082, tolerance: 0.004935986429691189
  model = cd_fast.enet_coordinate_descent_gram(
/Users/emmy/Desktop/Last_term_SMU/kaggle_comp/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09388301626173146, tolerance: 0.004935986429691189
  model = cd

In [39]:
model_1_df = pd.DataFrame(model_1.coef_, index = store_sku_part_trg.columns, columns=['Coefficient'] )
model_1_df[(model_1_df["Coefficient"] != 0)]

,Coefficient
Price,0.570930
Saleslag,-0.110409
Sales_mov_avg,0.461669
President_1,0.264711
Memorial_1,-0.238067


In [40]:
model_2_df = pd.DataFrame(model_2.coef_, index = store_compet_trg.columns, columns=['Coefficient'] )
model_2_df[(model_2_df["Coefficient"] != 0)]

,Coefficient


In [26]:
# (actual, full prediction * full residue) (actual, ols prediction * ols residue )

In [27]:
model_3 = ElasticNet(l1_ratio = 1)
model_3.positive = positive_features_1
model_3.fit(store_sku_part_trg, sku_sales_train)

benchmark_rsd = sku_sales_train - model_3.predict(store_sku_part_trg)

In [28]:
print(np.mean(np.exp(benchmark_rsd)))
print(np.mean(np.exp(sku_sales_overall_rsd)))

1.135305821675279
1.097447865363783


In [29]:
full_predict = model_1.predict(store_sku_part_test) + model_2.predict(store_compet_test)
print('MSE:', mean_squared_error(np.exp(sku_sales_test), np.exp(full_predict)*adjustment))
print('MAPE:', mean_absolute_percentage_error(np.exp(sku_sales_test), np.exp(full_predict)*adjustment))
print('MAE:', mean_absolute_error(np.exp(sku_sales_test), np.exp(full_predict)*adjustment))

MSE: 35.41063419154284
MAPE: 1.366798139896272
MAE: 5.1014358438033875


In [30]:
print('MSE:', mean_squared_error(np.exp(sku_sales_test), np.exp(full_predict)))
print('MAPE:', mean_absolute_percentage_error(np.exp(sku_sales_test), np.exp(full_predict)))
print('MAE:', mean_absolute_error(np.exp(sku_sales_test), np.exp(full_predict)))

MSE: 26.64201170450622
MAPE: 1.1801643048592292
MAE: 4.337613996082772


In [31]:
benchmark_predict = model_3.predict(store_sku_part_test)
print('MSE:', mean_squared_error(np.exp(sku_sales_test), np.exp(benchmark_predict)))
print('MAPE:', mean_absolute_percentage_error(np.exp(sku_sales_test), np.exp(benchmark_predict)))
print('MAE:', mean_absolute_error(np.exp(sku_sales_test), np.exp(benchmark_predict)))

MSE: 40.147329467552304
MAPE: 1.5028804906323692
MAE: 5.668256855481581


In [ ]:
def mean_percentage_error(y_true, y_pred):
    """
    Compute the mean percentage error (MPE) between y_true and y_pred.
    
    Parameters:
    y_true : array-like of shape (n_samples,)
        The true target values.
        
    y_pred : array-like of shape (n_samples,)
        The predicted target values.
        
    Returns:
    mpe : float
        The mean percentage error between y_true and y_pred.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean((y_true - y_pred) / y_true) * 100